In [1]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from bson.son import SON
from bson.code import Code
import json
import pandas as pd
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
# These are packages need for natural language processing:
import nltk
from __future__ import division, unicode_literals 
from textblob import TextBlob as tb
import math
from datetime import datetime
from dateutil import parser

In [2]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import amazon_data_sort 
from amazon_data_sort import get_b_product, get_reviews
import doc_summarization as ds

In [3]:
# #For Topic Modelling LDA:
# import lda
# #import scipy
# import gensim

# # from gensim import corpora, models, similarities
# # from gensim.models import hdpmodel, ldamodel
# # from itertools import izip

In [4]:
import reviews as r
import product_user_profile as pup
import doc_summarization as ds
import amazon_data_sort as ads

In [5]:
client = MongoClient()
mbf = client.dsbc.mbf
product_info = client.dsbc.product_info
name = product_info.find({}, {"_id":0,"asin":1,"title":1,"price":1})
beauty_list  = {}
for id_num in name:
    try:
        beauty_list[id_num["asin"]]=[id_num["title"],id_num["price"]]
    except:
        pass
#Creating database for reviews beauty file
rbf = client.dsbc.rbf

In [6]:
#These codes help find all sentence with the word of interests
#It's help full to see the exact condition surrounding the word
#to built all the terms for a category
from nltk import tokenize
for index in xrange(10):
    a = r.reviews(beauty_list,index)['Reviews']
    for i in a:
        sentences  = tokenize.sent_tokenize(i[0])
        for sentence in sentences:
            if "sun" in sentence:
                print sentence
                print ""
                print ""

As much as any of us hate to admit, how well we age is determined predominately by genetics plus exercise, good nutrition, hydration, sunscreen and regular skin care.


I will tell you, using this and staying out of the sun have protected my facial skin so much that, at age 56, I still have no wrinkles unless I smile really hard and you look up close.


"If you use this cream every day and stay out of the sun faithfully and/or use sunscreen, you will have beautiful skin long after your friends have started to get very noticeable lines.Regards,Karen


One of the few I've tried that doesn't smell like it contains sunscreen.


It's tint-free, which I also like.I use it primarily on my face and lower arms where the sun hits.




In [8]:
pup.product_profiler(r.reviews(beauty_list,5))

{'acne_score': 0.0,
 'anti_aging_score': 0.0,
 'combination_skin_score': 0.0,
 'dry_skin_score': 0.0,
 'oily_skin_score': 0.0,
 'redness_score': 0.0,
 'sensitive_score': 0.0,
 'sunscreen_score': 0.0}

###Now transfer everything onto MongoDB

In [17]:
item_profile = client.dsbc.item_profile

In [6]:
def summarize_item(item_list,item_index):
    all_item_info = {}
    all_item_info = {"item_id": item_list.keys()[item_index],"name":item_list.values()[item_index][0],
                     "price":item_list.values()[item_index][1]}
    all_item_score = pup.product_profiler(r.reviews(item_list,item_index))
    all_item_info.update(all_item_score)
    return all_item_info

In [7]:
summarize_item(beauty_list, 45)

{'acne_score': 0.04477611940298507,
 'anti_aging_score': 0.0,
 'combination_skin_score': 0.0,
 'dry_skin_score': 0.0845771144278607,
 u'item_id': u'B002D48QRK',
 u'name': u'Glytone Body Lotion, 8.4-Ounce Package',
 'oily_skin_score': 0.0,
 u'price': 25.22,
 'redness_score': 0.05970149253731343,
 'sensitive_score': 0.04975124378109453,
 'sunscreen_score': 0.0}

In [21]:
# These code is to save all item information 
# into mongodb called item_profile

# for i in xrange(5570):
#     beauty_item = summarize_item(beauty_list, i)
#     item_profile.insert_one(beauty_item)

In [15]:
# sp = client.dsbc.smarter_profile
# for i in xrange(5570):
#      beauty_item = summarize_item(beauty_list, i)
#      sp.insert_one(beauty_item)

In [103]:
x = smarter_profile.find({"dry_skin_score": {"$gt": 0.04}, "oily_skin_score":{"$gt":0.01}})
list_b = []
for i in xrange(1):
    list_b.append(x.sort("dry_skin_score", pymongo.DESCENDING)[i])

list_b

[{u'_id': ObjectId('558332463cae551ac7dfcab5'),
  u'acne_score': 0.25812619502868067,
  u'anti_aging_score': 0.04780114722753346,
  u'combination_skin_score': 0.02676864244741874,
  u'dry_skin_score': 0.3652007648183556,
  u'item_id': u'B000YJ2SLG',
  u'name': u'CeraVe Moisturizing Lotion, 12-Ounce Bottle',
  u'oily_skin_score': 0.040152963671128104,
  u'price': 12.78,
  u'redness_score': 34.0,
  u'sensitive_score': 0.26577437858508607,
  u'sunscreen_score': 0.022944550669216062}]

In [9]:
def list_of_10(priority,concern_list):
    dd ={"dry_skin_score":-0.1,"oily_skin_score":-0.1, 
         "sensitive_score":-0.1,"sunscreen_score":-0.1,
         "combination_skin_score":-0.1,"redness_score":-0.1,
         "anti_aging_score":-0.1,"acne_score":-0.1}

# concern_list = ["dry_skin_score", "acne_score"]
    for concern in concern_list:
        dd[concern]=0.0
    
    sp = MongoClient().dsbc.smarter_profile
    x = sp.find({"dry_skin_score":{"$gt":dd["dry_skin_score"]}, "oily_skin_score":{"$gt":dd["oily_skin_score"]},
                 "sensitive_score":{"$gt":dd["sensitive_score"]},"sunscreen_score":{"$gt":dd["sunscreen_score"]},
                 "combination_skin_score":{"$gt":dd["combination_skin_score"]}, "redness_score":{"$gt":dd["redness_score"]},
                 "anti_aging_score":{"$gt":dd["anti_aging_score"]},"acne_score":{"$gt":dd["acne_score"]}})
    number_of_items = min(10,x.count())
    list_of_items = []
    for i in xrange(number_of_items):
        list_of_items.append(x.sort(priority,pymongo.DESCENDING)[i])
    
    return list_of_items       

In [11]:
list_of_5("dry_skin_score",["dry_skin_score", "oily_skin_score","acne_score"])

10


[{u'_id': ObjectId('558332463cae551ac7dfcab5'),
  u'acne_score': 0.25812619502868067,
  u'anti_aging_score': 0.04780114722753346,
  u'combination_skin_score': 0.02676864244741874,
  u'dry_skin_score': 0.3652007648183556,
  u'item_id': u'B000YJ2SLG',
  u'name': u'CeraVe Moisturizing Lotion, 12-Ounce Bottle',
  u'oily_skin_score': 0.040152963671128104,
  u'price': 12.78,
  u'redness_score': 34.0,
  u'sensitive_score': 0.26577437858508607,
  u'sunscreen_score': 0.022944550669216062},
 {u'_id': ObjectId('55832fc13cae551ac7dfc7d2'),
  u'acne_score': 0.1724137931034483,
  u'anti_aging_score': 0.11494252873563218,
  u'combination_skin_score': 0.034482758620689655,
  u'dry_skin_score': 0.3620689655172414,
  u'item_id': u'B001ECQ4R6',
  u'name': u"POND'S Dry Skin Cream, 10.1-oz. (Pack of 3)",
  u'oily_skin_score': 0.08045977011494253,
  u'price': 17.03,
  u'redness_score': 8.0,
  u'sensitive_score': 0.06321839080459771,
  u'sunscreen_score': 0.0},
 {u'_id': ObjectId('55832ee43cae551ac7dfc6d4'),

In [104]:
x = sp.find({"dry_skin_score":{"$gt":0.0}, "oily_skin_score":{"$gt":-0.1},
                 "sensitive_score":{"$gt":0.0},"sunscreen_score":{"$gt":-0.1},
                 "combination_skin_score":{"$gt":-0.1}, "redness_score":{"$gt":-0.1},
                 "anti_aging_score":{"$gt":-0.1},"acne_score":{"$gt":-0.1}})
for i in x:
    print i

{u'dry_skin_score': 0.03389830508474576, u'oily_skin_score': 0.01694915254237288, u'name': u'Nu Skin Napca Moisturizer', u'sunscreen_score': 0.01694915254237288, u'sensitive_score': 0.03389830508474576, u'price': 19.97, u'combination_skin_score': 0.0, u'item_id': u'B004I0DQ0E', u'_id': ObjectId('558325373cae551ac7dfbb88'), u'redness_score': 0.0, u'anti_aging_score': 0.05084745762711865, u'acne_score': 0.0}
{u'dry_skin_score': 0.05555555555555555, u'oily_skin_score': 0.0, u'name': u'Nature Republic Super Aqua Max Moisture Watery Cream_80ml_for dry skin type', u'sunscreen_score': 0.0, u'sensitive_score': 0.046296296296296294, u'price': 12.1, u'combination_skin_score': 0.009259259259259259, u'item_id': u'B007VL84H2', u'_id': ObjectId('558325393cae551ac7dfbb8a'), u'redness_score': 0.0, u'anti_aging_score': 0.0, u'acne_score': 0.0}
{u'dry_skin_score': 0.024, u'oily_skin_score': 0.0, u'name': u'Neocutis Journee Biorestorative Day Cream with Psp, 0.5-Fluid Ounces', u'sunscreen_score': 0.104, 

In [102]:
list_of_3("dry_skin_score",["dry_skin_score","oily_skin_score"])

{u'dry_skin_score': 0.0, u'oily_skin_score': 0.0, u'combination_skin_score': -0.1, u'sunscreen_score': -0.1, u'sensitive_score': -0.1, u'redness_score': -0.1, u'anti_aging_score': -0.1, u'acne_score': -0.1}


[]

In [83]:
x = sp.find({"dry_skin_score": {"$gt": 0.04}, "oily_skin_score":{"$gt":0.01}})
for i in x.sort("dry_skin_score", pymongo.DESCENDING):
    print i
    print ""

{u'dry_skin_score': 0.3652007648183556, u'oily_skin_score': 0.040152963671128104, u'name': u'CeraVe Moisturizing Lotion, 12-Ounce Bottle', u'sunscreen_score': 0.022944550669216062, u'sensitive_score': 0.26577437858508607, u'price': 12.78, u'combination_skin_score': 0.02676864244741874, u'item_id': u'B000YJ2SLG', u'_id': ObjectId('558332463cae551ac7dfcab5'), u'redness_score': 34.0, u'anti_aging_score': 0.04780114722753346, u'acne_score': 0.25812619502868067}

{u'dry_skin_score': 0.3620689655172414, u'oily_skin_score': 0.08045977011494253, u'name': u"POND'S Dry Skin Cream, 10.1-oz. (Pack of 3)", u'sunscreen_score': 0.0, u'sensitive_score': 0.06321839080459771, u'price': 17.03, u'combination_skin_score': 0.034482758620689655, u'item_id': u'B001ECQ4R6', u'_id': ObjectId('55832fc13cae551ac7dfc7d2'), u'redness_score': 8.0, u'anti_aging_score': 0.11494252873563218, u'acne_score': 0.1724137931034483}

{u'dry_skin_score': 0.34328358208955223, u'oily_skin_score': 0.014925373134328358, u'name': u

In [55]:
#This function return a dictionary including item name, summary of all reviews
#for that item and items rating average
def reviews_rating_summary(item_list, item_index):
    item_info = {"item_id": item_list.keys()[item_index],"name":item_list.values()[item_index][0]}
    item = ads.get_b_product(item_list, item_index)
    reviews_sum = ds.summarize(item['Reviews'])
    
    item_info['reviews_summary'] = reviews_sum['top_n_summary']
    
    star_rating  = ads.strict_rating(item['rating']['One_star'],
                                     item['rating']['Two_star'],
                                     item['rating']['Three_star'],
                                     item['rating']['Four_star'],
                                     item['rating']['Five_star'],
                                     3, 100)
    len_reviews = sum(item['rating'].values())
    
    item_info['number_of_reviews'] = len_reviews
    item_info['star_rating_ave'] = star_rating
    return item_info

In [56]:
reviews_rating_summary(beauty_list, 0)

{u'item_id': u'B004I9J816',
 u'name': u'H2O Aquafirm Micro-Collagen Moisturizer Unisex, 1.7 Ounce',
 u'number_of_reviews': 2,
 u'reviews_summary': [u'this one seems like it might be a winner.',
  u'it absorbed nicely and my skin did look and feel great after using it.',
  u"it did lose one star, though, because of the price- it's a bit steep.i bought it for a collagen moisturizer and immediately found it cleared my breakouts.",
  u'i still use an under eye cream since i am 56 but this cream is miraculous.',
  u"it soaks in so fast you can't believe it and it not greasy or shiny for someone with oily skin."],
 u'star_rating_ave': 3.0294117647058822}

In [52]:
#Create a mongo database for the item's review info
item_reviews = client.dsbc.item_reviews

In [ ]:
cursor = item

In [49]:
# Put all item_reviews on mongo database:

# for item_index in xrange(5570):
#     item_review_info = reviews_rating_summary(beauty_list, item_index)
#     item_reviews.insert_one(item_review_info)

/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:83: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [9]:
def skin_type_counter(list_of_text):
    dry_skin = 0
    oily_skin = 0
    combination_skin = 0
    sensitivity = 0
    total_reviews = len(list_of_text)
    for review_text in list_of_text:
        new_text = review_text[0].lower()
        dry_skin += new_text.count("dry skin")
        oily_skin += new_text.count("oily skin")
        combination_skin += new_text.count("combination skin")
        sensitivity += new_text.count("sensitive")*tb(new_text).sentiment.polarity
    return [dry_skin, oily_skin, combination_skin, sensitivity/(total_reviews+1)]
    

In [11]:
a = get_reviews(beauty_list,42)['Reviews']
# for i in a:
#     print i
#     print skin_type_counter([i])
#     print ""
#     print ""

In [15]:
words = []
sentences  = nltk.word_tokenize(a[0][0])

In [28]:
stopwords = nltk.corpus.stopwords.words("english")\
            + ['usps','shipping','$','#', 'very', 'amazon','every',
                '&','order','seller','packaging','manufacture', 'ever',
                'package','cute','delivery', '.',',',"return", "might"
                '?','...','day','week', 'weeks']


In [29]:
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    text = [word for word in tokens if word not in stopwords]
    tagged_text = nltk.pos_tag(text)
    for word, tag in tagged_text:
        words.append({"word": word, "pos": tag})

In [32]:
from nltk.stem.wordnet import WordNetLemmatizer

In [41]:
nouns = []
lem = WordNetLemmatizer()
noun_only = [noun for noun in words if noun["pos"] in ["NN", "NNS"]]
for noun_word in noun_only:
    nouns.append(lem.lemmatize(noun_word["word"]))
    

In [44]:
import logging
import gensim
from gensim import corpora
from gensim.corpora import BleiCorpus

In [18]:
a = [1]+[2]+[2,3]

In [20]:
list(set(a))

[1, 2, 3]